# Extract the data from csv

In [6]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt(r'C:\Users\kurbi\Desktop\Data Science\Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

# Balance the dataset

In [12]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

# Standardize the inputs

In [15]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle

In [18]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split the dataset into train, validation and test

In [19]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1770.0 3579 0.4945515507124895
247.0 447 0.5525727069351231
220.0 448 0.49107142857142855


# Save the three datasets in *.npz

In [20]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs = test_inputs, targets = test_targets)

# Create the machine learning algorithm

In [5]:
import tensorflow as tf
import numpy as np

In [8]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)


# Model

In [14]:
# Model parameters
input_size = 10
hidden_layer_size = 50
output_size = 2

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=2)

Epoch 1/100
36/36 - 5s - 127ms/step - accuracy: 0.7139 - loss: 0.5520 - val_accuracy: 0.7673 - val_loss: 0.4866
Epoch 2/100
36/36 - 0s - 10ms/step - accuracy: 0.7513 - loss: 0.4543 - val_accuracy: 0.8009 - val_loss: 0.4278
Epoch 3/100
36/36 - 0s - 12ms/step - accuracy: 0.7737 - loss: 0.4112 - val_accuracy: 0.8188 - val_loss: 0.3897
Epoch 4/100
36/36 - 1s - 17ms/step - accuracy: 0.7969 - loss: 0.3852 - val_accuracy: 0.8121 - val_loss: 0.3712
Epoch 5/100
36/36 - 1s - 18ms/step - accuracy: 0.8053 - loss: 0.3696 - val_accuracy: 0.8009 - val_loss: 0.3633
Epoch 6/100
36/36 - 1s - 39ms/step - accuracy: 0.8106 - loss: 0.3569 - val_accuracy: 0.8098 - val_loss: 0.3502
Epoch 7/100
36/36 - 0s - 10ms/step - accuracy: 0.8055 - loss: 0.3498 - val_accuracy: 0.8300 - val_loss: 0.3373
Epoch 8/100
36/36 - 0s - 10ms/step - accuracy: 0.8117 - loss: 0.3456 - val_accuracy: 0.8054 - val_loss: 0.3474
Epoch 9/100
36/36 - 0s - 11ms/step - accuracy: 0.8184 - loss: 0.3423 - val_accuracy: 0.8233 - val_loss: 0.3378


In [17]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss: {:.2f}. Test accuracy: {:.2f}%'.format(test_loss, test_accuracy * 100))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8013 - loss: 0.3647
Test loss: 0.36. Test accuracy: 80.13%
